# Cable Sizing

In [1]:
from python_electric import Q_
from python_electric.network import (
    Load,
    Cable,
    ConductorMaterial,
    InsulationMaterial,
    InstallMethod,
    CableMounting,
    CableArrangement
)

## Load

In [2]:
load = Load(
    U_l=Q_(400, 'V'),
    cos_phi=0.8,
    P_e=Q_(100, 'kW')
)

print(f"load current: {load.I_b.to('A'):~P.0f}")

load current: 180 A


## Cable Sizing

**installation method**<br>
This parameter is used to determine the cross-sectional area $S$ of the conductors and the ampacitity $I_{z0}$ of the cable at standard reference conditions.

- `A1`<br>
  Insulated conductors in a conduit in a thermally insulated wall.
- `A2`<br>
  Multicore cables in a conduit in a thermally insulated wall.
- `B1`<br>
  Insulated conductors in a conduit against a surface (wooden or stone wall).
- `B2`<br>
  Multicore cables in a conduit against a surface (wooden or stone wall).
- `C`<br>
  Single or multicore cables fixed on (or spaced less than 0.3x the cable diameter from) a surface (wooden or stone wall), or an unperforated tray.
- `D`<br>
  Single or multicore cables in a conduit (or in cable ducting) in the ground.
- `E`<br>
  Multicore cable on a horizontal/vertical perforated cable tray or ladder – Multicore cable in mounting brackets with sufficient space between the cable and the surface (at least 0.3 x the cable diameter).
- `F`<br>
  Insulated conductors, touching each other, on a horizontal/vertical perforated cable tray or ladder – Insulated conductors in mounting brackets with sufficient space between the conductors and the wall (at least 0.3 x the conductor diameter).

**cable mounting**<br>
This parameter is used to determine the correction factor that takes into account the influence of nearby circuits.

- `BUNCHED_SIMILAR`<br>
  Bunched in air, on a surface, embedded or enclosed; cables have similar sections or loads.
- `UNPERFORATED_TRAY`<br>
  Single layer on wall, floor, or unperforated tray.
- `CEILING`<br>
  Single layer fixed directly under a (wooden) ceiling.
- `PERFORATED_TRAY`<br>
  Single layer on a perforated horizontal or vertical cable tray.
- `LADDER`<br>
  Single layer on ladder support or cleats, etc.
- `BUNCHED_DISSIMILAR`<br>
  Bunched in air, on a surface, embedded or enclosed; cables have dissimilar sections or loads.

> *Number of nearby circuits*<br>
Multiple single-core cables forming a single three-phase circuit do not require correction of the current-carrying capacity (ampacity) of the cable. For example, if a consumer is supplied with four single-core cables (three phases and one neutral), no correction is required.
However, if multiple single-core cables are laid in parallel to supply a consumer, these cables are considered multiple circuits. For example, if a consumer is supplied with two parallel multicore cables or with six single-core cables, with two cables per phase, the correction factor corresponding to two circuits is used.


**cable arrangment**<br>
This parameter is used to determine conductor reactance.

- `BUS_BAR`<br>
  Conductors are bus bars.
- `MULTICORE`<br>
  Conductors in multicore cable.
- `SINGLE_CORE_SPACED`<br>
  Single core cables/conductors, separated from each other.
- `SINGLE_CORE_SPACED_2R`<br>
  Single core cables/conductors; spacing distance at least one diameter.
- `SINGLE_CORE_SPACED_4R`<br>
  Single core cables/conductors; spacing distance at least two diameters.
- `SINGLE_CORE_TREFOIL`<br>
  Single core cables/conductors; grouped in clover.
- `SINGLE_CORE_TOUCHING`<br>
  Single core cables/conductors, touching each other

## Determine the Conductor Cross-Sectional Area

A cable is sized when the `Cable` class is instantiated. Depending on the load, multiple parallel conductors or single-core cables may be obtained per phase.

In [3]:
cable1 = Cable(
    name="cable1",
    U_l=load.U_l,
    I_b_ph=load.I_b,
    cos_phi=load.cos_phi,
    L=Q_(10, 'm'),
    k_simul=1.0,
    k_ext=1.0,
    conductor_material=ConductorMaterial.COPPER,
    insulation_material=InsulationMaterial.XLPE,
    T_amb=Q_(40, 'degC'),
    install_method=InstallMethod.E,
    cable_mounting=CableMounting.BUNCHED_SIMILAR,
    cable_arrangement=CableArrangement.MULTICORE,
    num_other_circuits=4,
    h3_fraction=0.0
)

print(
    f"load current per phase: {cable1.I_b_ph.to('A'):~P.0f}",
    f"compensated load current per phase: {cable1.I_bc_ph.to('A'):~P.0f}",
    f"single conductor load current: {cable1.I_b.to('A'):~P.0f}",
    f"number of conductors per phase: {cable1.n_phase}",
    f"single conductor cross-sectional area: {cable1.S.to('mm**2'):~P.0f}",
    f"single conductor standard current-carrying capacity: {cable1.I_z0.to('A'):~P.1f}",
    f"single conductor actual current-carrying capacity: {cable1.I_z.to('A'):~P.1f}",
    sep="\n"
)

load current per phase: 180 A
compensated load current per phase: 180 A
single conductor load current: 180 A
number of conductors per phase: 1
single conductor cross-sectional area: 150 mm²
single conductor standard current-carrying capacity: 395.0 A
single conductor actual current-carrying capacity: 215.7 A


> Note: The compensated load current takes the simultaneity factor `k_simul` and the expansion factor `k_ext` into account. Conductors are sized based on the compensated load current.

## Checking the Conductor Cross-Sectional Area

If the conductor cross-sectional area is specified, a check is made when the `Cable` class is created to determine whether the cross-section is sufficient. If too small, a `ValueError` exception is raised.

In [4]:
cable2 = Cable(
    name="cable2",
    U_l=load.U_l,
    I_b_ph=load.I_b,
    cos_phi=load.cos_phi,
    L=Q_(10, 'm'),
    k_simul=1.0,
    k_ext=1.0,
    conductor_material=ConductorMaterial.COPPER,
    insulation_material=InsulationMaterial.XLPE,
    T_amb=Q_(40, 'degC'),
    install_method=InstallMethod.E,
    cable_mounting=CableMounting.BUNCHED_SIMILAR,
    cable_arrangement=CableArrangement.MULTICORE,
    num_other_circuits=4,
    h3_fraction=0.0,
    S=Q_(185, 'mm**2')
)

print(
    f"load current per phase: {cable2.I_b_ph.to('A'):~P.0f}",
    f"compensated load current per phase: {cable2.I_bc_ph.to('A'):~P.0f}",
    f"single conductor load current: {cable2.I_b.to('A'):~P.0f}",
    f"number of conductors per phase: {cable2.n_phase}",
    f"single conductor cross-sectional area: {cable2.S.to('mm**2'):~P.0f}",
    f"single conductor standard current-carrying capacity: {cable2.I_z0.to('A'):~P.1f}",
    f"single conductor actual current-carrying capacity: {cable2.I_z.to('A'):~P.1f}",
    sep="\n"
)

load current per phase: 180 A
compensated load current per phase: 180 A
single conductor load current: 180 A
number of conductors per phase: 1
single conductor cross-sectional area: 185 mm²
single conductor standard current-carrying capacity: 450.0 A
single conductor actual current-carrying capacity: 245.7 A
